In [1]:
import numpy as np
import scipy.io as scio
import pandas as pd

from sklearn.cluster import KMeans
from sklearn import metrics

import VFC_dyad 
import VnFC_ex2fd as cal_VFC
import Idiff
import Idiff_bootstrap

### 输入输出和参数设置

输入数据

In [2]:
# 输入数据路径
GROUP1_DATA2021_FOLDER = 'data2021/Class1-Parent'#亲子组
GROUP2_DATA2021_FOLDER = 'data2021/Class2-Stranger'#陌生人组
GROUP1_DATA2023_FOLDER = 'data2023/Class1-Parent'#亲子组
GROUP2_DATA2023_FOLDER = 'data2023/Class2-Stranger'#陌生人组
GROUP1_DATAALL_FOLDER = 'data_all/Class1-Parent'#亲子组
GROUP2_DATAALL_FOLDER = 'data_all/Class2-Stranger'#陌生人组

# 输入数据结构：
# 每个数据文件包含6种拼图场景，其中前面的1/2表示儿童拼或者成人拼，
# 后面的1/2/3表示单独/陪伴/合作,只关注3（合作）
sit_c1 = 'ca11' #儿童拼-单独
sit_a1 = 'ca21' #成人拼-单独
sit_c2 = 'ca12' #儿童拼-陪伴
sit_a2 = 'ca22' #成人拼-陪伴
sit_c3 = 'ca13' #儿童拼-合作
sit_a3 = 'ca23' #成人拼-合作
# 每种情况包含4个cell文件，分别对应4个频段：1-3，3-6，6-9，9-12
delta,theta,alpha,belta=0,1,2,3
freq_all=['delta','theta','alpha','belta']
# 每个cell文件的格式为通道数*时间点数*被试数
# 通道数: 其中1-28为儿童，29-56为成人，通过subc选择类型
# 28个通道及其对应的索引为：
channame_full = ['Fp1','Fp2','Fz','F3','F4','F7','F8','FC1','FC2','FC5','FC6','Cz','C3','C4','T7','T8','CP1','CP2','CP5','CP6','Pz','P3','P4','P7','P8','Oz','O1','O2']
id_chan_all = list(range(0,28))
# 26个通道及其对应的索引为：
channame_test = ['Fp1','Fp2','Fz','F3','F4','F7','F8','FC1','FC2','FC5','FC6','Cz','C3','C4','CP1','CP2','CP5','CP6','Pz','P3','P4','P7','P8','Oz','O1','O2']
id_chan_test = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

参数设置

In [3]:
# 参数设置
chan_num = 28 #通道数
edge_num = int(chan_num*(chan_num-1)/2) #边数
Nuptran_indices = VFC_dyad.uptran_mask(chan_num) #矩阵上三角列向量的位置
Euptran_indices = VFC_dyad.uptran_mask(edge_num) #矩阵上三角列向量的位置
Vtype='all'

# itype指边连接强度的计算方式，'p','c','d'分别代表：PLV复指数、corr相关、delta做差
itype_all=['e','c','d']
itype=itype_all[0]
# corr_type指边与边之间共波动特性的计算方式，'cos','pearson','delta'分别代表：
# 余弦相似度，皮尔逊相关系数，直接做差
corr_type_all=['cos','pearson','delta']
corr_type=corr_type_all[0]

tp=2000

sit=sit_a3 #拼图场景
freq=theta #频段
subc='adult'  #对象类型 

### 计算总体指标 Idiff_dyad

#### EFC矩阵对应的 Idiff_subj

In [4]:
# 比较eFC矩阵对于某种模态mode的识别能力
# 将所有模态对应的index拼接在一起
dyad_all1 = Idiff.file_path_infd(GROUP1_DATAALL_FOLDER)
VeFC_mode, trn_mode = VFC_dyad.VeFC_dyad_all(dyad_all1,sit,freq,subc,Vtype,Euptran_indices,corr_type,tp)

dyad_prt = dyad_all1
VeFC_prt = VeFC_mode
trn_prt = trn_mode
I_diff,I_self,I_other,I_self_mean,per_change,per_diff=Idiff.cal_Iden(VeFC_mode,trn_mode,'pearsonr')
print('%.2f'%I_diff,'%.2f'%I_self_mean,'%.2f'%I_other)
# 保留两位小数并记录数据
I_concat = [I_diff,I_self_mean,I_other,per_change,per_diff]
I_eFC1 = np.around(I_concat,2)

8.64 0.59 0.51


计算聚类准确率

In [5]:
# 对Vefc向量做Kmeans，然后计算聚类准确率相关的指标
def Kmeans_score(dyad,VeFC,trn):
    # 做kmeans得到预测标签值
    kmeans = KMeans(n_clusters=dyad.shape[0], random_state=0, n_init=10).fit(VeFC.T)

# 拼接得到真实标签值
    i=0
    true_labels=[]
    for i in range(dyad.shape[0]):
        true_label=[i]*trn[0][i]
        if i==0:
            true_labels=true_label
            i=1
        else:
            true_labels=np.r_[true_labels,true_label]
    true_labels = np.array(true_labels) 

# 计算预测标签和真实标签吻合程度的评价指标
    rand_score = metrics.adjusted_rand_score(true_labels, kmeans.labels_) #兰德系数 
    mi_score = metrics.adjusted_mutual_info_score(true_labels, kmeans.labels_) #互信息指数
    fm_score = metrics.fowlkes_mallows_score(true_labels, kmeans.labels_) #FM分数
    print('%.2f'%rand_score,'%.2f'%mi_score,'%.2f'%fm_score)

# 亲子组
Kmeans_score(dyad=dyad_prt,VeFC=VeFC_prt,trn=trn_prt)
# # 陌生人组
# Kmeans_score(dyad=dyad_stg,VeFC=VeFC_stg,trn=trn_stg)

0.34 0.53 0.36


#### NFC矩阵对应的Idiff_subj

In [10]:
# 比较eFC矩阵对于某种模态mode的识别能力
# 将所有模态对应的index拼接在一起
dyad_all1 = Idiff.file_path_infd(GROUP1_DATAALL_FOLDER)
VnFC_mode, trn_mode = VFC_dyad.VnFC_dyad_all(dyad_all1,sit,freq,subc,Vtype,Nuptran_indices,tp)

dyad_prt = dyad_all1
VeFC_prt = VnFC_mode
trn_prt = trn_mode
I_diff,I_self,I_other,I_self_mean,per_change,per_diff=Idiff.cal_Iden(VnFC_mode,trn_mode,'pearsonr')
print('%.2f'%I_diff,'%.2f'%I_self_mean,'%.2f'%I_other)
# 保留两位小数并记录数据
I_concat = [I_diff,I_self_mean,I_other,per_change,per_diff]
I_nFC1 = np.around(I_concat,2)

6.47 0.79 0.73


计算聚类准确率

In [7]:
# 亲子组
Kmeans_score(dyad=dyad_prt,VeFC=VeFC_prt,trn=trn_prt)
# # 陌生人组
# Kmeans_score(dyad=dyad_stg,VeFC=VeFC_stg,trn=trn_stg)

0.40 0.58 0.42


#### 导出结果至excel文件

In [8]:
# # 输出数据所要导出到的Excel文件
# output_excel='output/Diff_dyad_all.xlsx'
# # df=pd.DataFrame() #构造原始数据文件
# # df.to_excel(output_excel)

# # 创建一个新的工作表，命名为：场景+对象类型+频段
# new_sheet_name = sit+'-'+subc+'-'+freq_all[freq] 
# # # 比较各个分支的结果
# # new_sheet_name = itype+'-'+corr_type
# excel_writer = pd.ExcelWriter(output_excel, mode='a',engine='openpyxl')
# # 拼接所有条件下的I_concat数据
# I_all = np.c_[I_eFC1,I_nFC1]
# I_all = I_all.T

# # 转换为DataFrame格式
# df_Iall = pd.DataFrame(I_all)
# df_Iall.columns=['I_diff','I_self','I_other','Change_rate','Diff_rate'] #设置列名
# # 只比较亲子组
# df_Iall.index=['eFC','nFC']
# # # 两个组的比较
# # df_Iall.index=['Group1-eFC','Group1-nFC','Group2-eFC','Group2-nFC'] #设置索引

# # 保存至excel
# df_Iall.to_excel(excel_writer, sheet_name=new_sheet_name)
# excel_writer.save()

### 统计检验

#### Bootstrapping
使用Bootstrapping方法进行重抽样：30% = 15组，1000次

In [9]:
I_eFC_all,I_nFC_all = Idiff_bootstrap.Idiff_boot(res_time=1000,yita=0.3,VeFC_mode=VeFC_mode,VnFC_mode=VnFC_mode,
         trn_mode=trn_mode)

In [10]:
# 转换为DataFrame格式
I_eFC_allt=I_eFC_all.T
df_eFC1 = pd.DataFrame(I_eFC_allt)
# 添加列名和索引
df_eFC1.columns=['I_diff','I_self','I_other','Change_rate','Diff_rate'] #设置列名

# 转换为DataFrame格式
I_nFC_allt=I_nFC_all.T
df_nFC1 = pd.DataFrame(I_nFC_allt)
# 添加列名和索引
df_nFC1.columns=['I_diff','I_self','I_other','Change_rate','Diff_rate'] #设置列名

##### 导出结果至excel文件

In [63]:
output_excel_stats = 'output/Diff_dyad_boot_all.xlsx'
# df=pd.DataFrame() #构造原始数据文件
# df.to_excel(output_excel_stats)

# 创建一个新的工作表，命名为：场景+对象类型+频段
new_sheet_name = sit+'-'+subc+'-'+freq_all[freq] 
# # 比较各个分支的结果
# new_sheet_name = itype+'-'+corr_type
excel_writer = pd.ExcelWriter(output_excel_stats, mode='a',engine='openpyxl')
'''df = pd.DataFrame({new_sheet_name: []})
df.to_excel(excel_writer, sheet_name=new_sheet_name, index=False)
excel_writer.save()'''
# 亲子组1个组的拼接
df_all = pd.concat([df_eFC1,df_nFC1],axis=1)

# 保存至excel
df_all.to_excel(excel_writer, sheet_name=new_sheet_name)
excel_writer.save()

'df = pd.DataFrame({new_sheet_name: []})\ndf.to_excel(excel_writer, sheet_name=new_sheet_name, index=False)\nexcel_writer.save()'

#### 两两比较
不同dyad两两比较，可以理解为重抽样：2组，2000次

In [47]:
i,j=1,0
# 二人组i,二人组j对应的trial数目
trn_i = trn_prt[:,i]
trn_j = trn_prt[:,j]
# 二人组i,二人组j的trial在VFC序列中对应的前后索引位置
pre_idx_i,post_idx_i=int(np.sum(trn_prt[:,:i])),int(np.sum(trn_prt[:,:i])+trn_i)
pre_idx_j,post_idx_j=int(np.sum(trn_prt[:,:j])),int(np.sum(trn_prt[:,:j])+trn_j)
VFC_mode_i = VeFC_mode[:,pre_idx_i:post_idx_i]
VFC_mode_j = VeFC_mode[:,pre_idx_j:post_idx_j]

trn_ij = np.c_[trn_i,trn_j]
VFC_mode_ij = np.c_[VFC_mode_i,VFC_mode_j]
VFC_mode_ij.shape

(142884, 26)

In [49]:
# dyad二人组数目
dyad_num = trn_prt.shape[1]
k=0

for i in range(1,dyad_num):
    for j in range(i):
        # 二人组i,二人组j对应的trial数目
        trn_i = trn_prt[:,i]
        trn_j = trn_prt[:,j]
        # 二人组i,二人组j的trial在VFC序列中对应的位置
        # 前后索引
        pre_idx_i,post_idx_i=int(np.sum(trn_prt[:,:i])),int(np.sum(trn_prt[:,:i])+trn_i)
        pre_idx_j,post_idx_j=int(np.sum(trn_prt[:,:j])),int(np.sum(trn_prt[:,:j])+trn_j)
        VFC_mode_i = VeFC_mode[:,pre_idx_i:post_idx_i]
        VFC_mode_j = VeFC_mode[:,pre_idx_j:post_idx_j]
        # 拼接二人组i,二人组j的trial数目、VFC序列
        trn_ij = np.c_[trn_i,trn_j]
        VFC_mode_ij = np.c_[VFC_mode_i,VFC_mode_j]
        # 计算二人组i,二人组j的识别性指标I_diff
        I_diff,I_self,I_other,I_self_mean,per_change,per_diff=Idiff.cal_Iden(VFC_mode_ij,trn_ij,'pearsonr')

        # 保留两位小数并记录数据
        I_concat = [I_diff,I_self_mean,I_other,per_change,per_diff]
        I_eFC1 = np.around(I_concat,2)
        # 将所有二人组对的指标拼接
        if k==0:
            I_eFC_all=I_eFC1
            k=1
        else:
            I_eFC_all=np.c_[I_eFC_all,I_eFC1]
            
# 转换为DataFrame格式
I_eFC_allt=I_eFC_all.T
df_eFC1 = pd.DataFrame(I_eFC_allt)
# 添加列名和索引
df_eFC1.columns=['I_diff','I_self','I_other','Change_rate','Diff_rate'] #设置列名

In [55]:
# dyad二人组数目
dyad_num = trn_prt.shape[1]
k=0

for i in range(1,dyad_num):
    for j in range(i):
        # 二人组i,二人组j对应的trial数目
        trn_i = trn_prt[:,i]
        trn_j = trn_prt[:,j]
        # 二人组i,二人组j的trial在VFC序列中对应的位置
        # 前后索引
        pre_idx_i,post_idx_i=int(np.sum(trn_prt[:,:i])),int(np.sum(trn_prt[:,:i])+trn_i)
        pre_idx_j,post_idx_j=int(np.sum(trn_prt[:,:j])),int(np.sum(trn_prt[:,:j])+trn_j)
        VFC_mode_i = VnFC_mode[:,pre_idx_i:post_idx_i]
        VFC_mode_j = VnFC_mode[:,pre_idx_j:post_idx_j]
        # 拼接二人组i,二人组j的trial数目、VFC序列
        trn_ij = np.c_[trn_i,trn_j]
        VFC_mode_ij = np.c_[VFC_mode_i,VFC_mode_j]
        # 计算二人组i,二人组j的识别性指标I_diff
        I_diff,I_self,I_other,I_self_mean,per_change,per_diff=Idiff.cal_Iden(VFC_mode_ij,trn_ij,'pearsonr')

        # 保留两位小数并记录数据
        I_concat = [I_diff,I_self_mean,I_other,per_change,per_diff]
        I_nFC1 = np.around(I_concat,2)
        # 拼接
        if k==0:
            I_nFC_all=I_nFC1
            k=1
        else:
            I_nFC_all=np.c_[I_nFC_all,I_nFC1]

# 转换为DataFrame格式
I_nFC_allt=I_nFC_all.T
df_nFC1 = pd.DataFrame(I_nFC_allt)
# 添加列名和索引
df_nFC1.columns=['I_diff','I_self','I_other','Change_rate','Diff_rate'] #设置列名

##### 导出结果至excel文件

In [60]:
output_excel_stats = 'output/Diff_dyad_btw2_all.xlsx'
# df=pd.DataFrame() #构造原始数据文件
# df.to_excel(output_excel_stats)

# 创建一个新的工作表，命名为：场景+对象类型+频段
new_sheet_name = sit+'-'+subc+'-'+freq_all[freq] 
# # 比较各个分支的结果
# new_sheet_name = itype+'-'+corr_type
excel_writer = pd.ExcelWriter(output_excel_stats, mode='a',engine='openpyxl')
'''df = pd.DataFrame({new_sheet_name: []})
df.to_excel(excel_writer, sheet_name=new_sheet_name, index=False)
excel_writer.save()'''

'df = pd.DataFrame({new_sheet_name: []})\ndf.to_excel(excel_writer, sheet_name=new_sheet_name, index=False)\nexcel_writer.save()'

In [61]:
# 亲子组1个组的拼接
df_all = pd.concat([df_eFC1,df_nFC1],axis=1)

# 保存至excel
df_all.to_excel(excel_writer, sheet_name=new_sheet_name)
excel_writer.save()

C:\Users\吴诚成\AppData\Local\Temp\ipykernel_12924\1829889429.py:6: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
